In [1]:
import numpy as np
from scipy import integrate, optimize
import matplotlib.pyplot as plt

For this problem, we will need to consider a differential element of the chain. In the rotating frame, the relevant forces are tension, gravity and the centrifugal force. We will parametrise the shape of the chain, with $x(s)$ representing the vertical coordinate and $r(s)$ representing the radial coordinate, and s representing the distance along the chain.
These variables will be related by the constraint $x'(s)^2 + r'(s)^2 = 1$, which means that a differential increase of x and r along the chain equals a differential increase in s. The tension in the chain will then be $T(s)$, with vertical component $T_x(s) = T(s) x'(s)$ and radial component $T_r(s) = T(s) r'(s)$. We will denote the linear mass density of the chain $\mu$ and its angular velocity about the vertical $\omega$.

Balancing forces along the chain:
In x direction: $T_x'(s) = - \mu\, g$
In r direction: $T_r'(s) = - \mu\, \omega^2\, r(s)$

Solving for $x''(s)$ and $r''(s)$ in terms of $T(s)$ and its derivative, we get:
$$
\begin{align}
x''(s) &= - \frac{\mu\, g + T'(s)\, x'(s)}{T(s)} \\
r''(s) &= - \frac{\mu\, \omega^2\, r(s) + T'(s)\, r'(s)}{T(s)}
\end{align}
$$

Differentiating the constraint equation, and solving for $T'(s)$:
$$
\begin{align}
0 &= x'(s) x''(s) + y'(s) y''(s) \\
\ &= - \frac{r'(s) \left(\mu\, \omega^2\, r(s) + T'(s)\, r'(s) \right) + x'(s) \left(\mu\, g + T'(s)\, x'(s) \right)}{T(s)} \\
\\
T'(s) &= - \mu\, \omega^2\, r(s)\, r'(s) - \mu\, g\, x'(s)
\end{align}
$$

Substituting this into the expressions for $x''(s)$ and $r''(s)$, we can solve these 3 equations as a system of 5 ODEs as a boundary value problem. The boundary conditions are $x(0) = 0$, $r(0) = 0$, $T(l) = 0$ and $x'(0)$ is free to ensure that the last condition is met. However, we do not have enough boundary conditions for an exact solution. So, we need 1 more.

We will do this by integrating the force balance along x:
$$
\begin{align}
\int_{0}^{l} T_x'(s)\, ds &= \int_{0}^{l} - \mu\, g\, ds \\
T_x(l) - T_x(0) &= -\mu\, g\, l \\
T(0)\,x'(0) &= \mu\, g\, l \\
T(0) &= \frac{\mu\, g\, l}{x'(0)}
\end{align}
$$

Thus, we solve this as a boundary value problem with $x(0) = 0$, $r(0) = 0$, $x'(0)^2 + r'(0)^2 = 1$, $T(0) = \frac{\mu\, g\, l}{x'(0)}$, $T(l) = 0$

In [89]:
g = 9.81
mu = 0.1
omega = 10
l = 1

epsilon = 10E-5

def swinging_chain(s, vec):
    x, xp, r, rp, T = vec

    Tp = - mu * np.power(omega, 2) * r * rp - mu * g * xp

    return [
        xp,
        - (mu * g + Tp * xp) / T,
        rp,
        - (mu * np.power(omega, 2) * r + Tp * rp) / T,
        Tp
    ]

def bc(ya, yb):
    return [
        ya[0],
        ya[2],
        np.power(ya[1], 2) + np.power(ya[3], 2) - 1,
        ya[1] * ya[4] - mu * g * l,
        yb[4] - epsilon
    ]

s_values = np.linspace(0, l, 50)
initial_values = np.array([
    np.zeros(50),
    np.ones(50),
    np.zeros(50),
    np.zeros(50),
    np.concatenate([np.ones(25) * mu * g * l, np.ones(25) * epsilon])
])

sol = integrate.solve_bvp(swinging_chain, bc, s_values, initial_values)

C:\Users\kaloo\AppData\Local\Temp\ipykernel_5432\3272882904.py:15: RuntimeWarning: divide by zero encountered in divide
  - (mu * g + Tp * xp) / T,
C:\Users\kaloo\AppData\Local\Temp\ipykernel_5432\3272882904.py:17: RuntimeWarning: divide by zero encountered in divide
  - (mu * np.power(omega, 2) * r + Tp * rp) / T,
C:\Users\kaloo\AppData\Roaming\Python\Python39\site-packages\scipy\integrate\_bvp.py:311: RuntimeWarning: invalid value encountered in add
  col_res = y[:, 1:] - y[:, :-1] - h / 6 * (f[:, :-1] + f[:, 1:] +
C:\Users\kaloo\AppData\Roaming\Python\Python39\site-packages\scipy\integrate\_bvp.py:40: RuntimeWarning: invalid value encountered in subtract
  df_dy[:, i, :] = (f_new - f0) / hi
C:\Users\kaloo\AppData\Roaming\Python\Python39\site-packages\scipy\integrate\_bvp.py:38: RuntimeWarning: invalid value encountered in subtract
  hi = y_new[i] - y[i]
C:\Users\kaloo\AppData\Roaming\Python\Python39\site-packages\scipy\integrate\_bvp.py:566: RuntimeWarning: invalid value encountered

In [2]:
def swinging_chain(s, vec, mu, omega, g):
    x, xp, r, rp, T = vec

    Tp = - mu * np.power(omega, 2) * r * rp - mu * g * xp

    return [
        xp,
        - (mu * g + Tp * xp) / T,
        rp,
        - (mu * np.power(omega, 2) * r + Tp * rp) / T,
        Tp
    ]

g = 9.81
mu = 0.01
omega = 40
l = 1

# xdo_values = np.linspace(0.05, 1, 100)

def Tf(xdo):
    print(xdo)

    sol = integrate.solve_ivp(swinging_chain, [0, l], [0, xdo, 0, np.sqrt(1 - np.power(xdo, 2)), mu * g * l / xdo], args=[mu, omega, g], max_step = 0.005)

    return sol.y[4, -1] * sol.y[3, -1]

# Tf_values = np.array([Tf(xdo) for xdo in xdo_values])

In [144]:
sol = optimize.root_scalar(Tf, method="secant", x0=0.1, x1=0.2, bracket=(0.01, 0.9), rtol=0.01)

In [6]:
initial_values = np.linspace(0.01, 0.3, 100)

xdo_sols = [optimize.root_scalar(Tf, method="secant", x0=xdo_i, x1=xdo_i + 0.001, bracket=(0.01, 0.9), rtol=0.01) for xdo_i in initial_values]
print(xdo_sols)

0.01
0.011
0.012029284807158556
0.012219181633422073
0.01292929292929293
0.01392929292929293
0.012116740701543345
0.012292070316395589
0.015858585858585857
0.016858585858585858
0.009616477274883236
0.013566418388205053
0.012582314681215267
0.012208746724460814
0.018787878787878787
0.01978787878787879
0.0019306510144386983
0.018870933792150553
0.01804304463787392
0.004925589654675409
0.016102287017540066
0.014744218932562572
0.01084172084121905
0.01264296473970288
0.012310420364255607
0.021717171717171718
0.02271717171717172
-0.0190226127694711
0.03277537406893081
0.04547732504573937
0.10562663225871341
0.10862913605469471
0.024646464646464646
0.025646464646464646
-0.1011304612288645
0.07895644645391577
0.09837388769928451
0.1081470138364858
0.10963892318033613
0.027575757575757573
0.028575757575757574
0.8108468469073248
0.8250367384678915
0.2941284813772198
0.34011859183381554
0.29538171480718245
0.030505050505050507
0.03150505050505051
0.17290040746405042
0.14936863386183774
-0.160514

C:\Users\Luc\AppData\Local\Temp\ipykernel_15084\647133503.py:24: RuntimeWarning: invalid value encountered in sqrt
  sol = integrate.solve_ivp(swinging_chain, [0, l], [0, xdo, 0, np.sqrt(1 - np.power(xdo, 2)), mu * g * l / xdo], args=[mu, omega, g], max_step = 0.005)

KeyboardInterrupt



In [7]:
print(xdo_sols[0])

NameError: name 'xdo_sols' is not defined

In [62]:
g = 9.81
mu = 0.005
omega = 40
l = 0.5

xdo_i = 0.9
xdo = optimize.root_scalar(Tf, method="secant", x0=xdo_i, x1=xdo_i + 0.001, bracket=(0.01, 0.9), rtol=0.01).root
print(xdo)

# xdo_values = np.linspace(0.05, 1, 100)

sol = integrate.solve_ivp(swinging_chain, [0, l], [0, xdo, 0, np.sqrt(1 - np.power(xdo, 2)), mu * g * l / xdo], args=[mu, omega, g], max_step = 0.005)

plt.plot(sol.y[2], -sol.y[0])
plt.show()

data = []
for i in range(len(sol.y[0])):
    data.append([sol.y[0][i] + 0.01 * (np.random.random() - 0.5), sol.y[2][i] + 0.01 * (np.random.random() - 0.5)])

with open("data/swinging chain/chain 4/data 5.csv", "w") as f:
    np.savetxt(f, data, fmt="%.3f", delimiter=",")

0.9
0.901
-39.525093583366676


C:\Users\Luc\AppData\Local\Temp\ipykernel_15084\647133503.py:24: RuntimeWarning: invalid value encountered in sqrt
  sol = integrate.solve_ivp(swinging_chain, [0, l], [0, xdo, 0, np.sqrt(1 - np.power(xdo, 2)), mu * g * l / xdo], args=[mu, omega, g], max_step = 0.005)

KeyboardInterrupt

